In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize
import Outils

In [110]:
#importer les odnnées de  base
with ct.ConnexionBdd('local_otv') as c : 
    df = gp.read_postgis("select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.the_geom) as tgt_geom from public.test_agreg_lineaire t left join public.test_agreg_lineaire_vertices_pgr v1 on t.\"source\"=v1.id left join public.test_agreg_lineaire_vertices_pgr v2  on t.target=v2.id", c.connexionPsy)
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df.loc[:,'nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes
df=at.identifier_rd_pt(df)

In [71]:
toto=list(at.recup_troncon_elementaire('TRONROUT0000000032856211',df))

In [72]:
toto.append('TRONROUT0000000032856211')

In [74]:
toto

['TRONROUT0000000032856177',
 'TRONROUT0000000032856205',
 'TRONROUT0000000032852549',
 'TRONROUT0000000129676626',
 'TRONROUT0000000032856180',
 'TRONROUT0000000032856217',
 'TRONROUT0000000121809347',
 'TRONROUT0000000121809348',
 'TRONROUT0000000121809342',
 'TRONROUT0000000032856170',
 'TRONROUT0000000032856168',
 'TRONROUT0000000032856173',
 'TRONROUT0000000032856226',
 'TRONROUT0000000032856231',
 'TRONROUT0000000032856234',
 'TRONROUT0000000032852588',
 'TRONROUT0000000032852547',
 'TRONROUT0000000032852561',
 'TRONROUT0000000032852614',
 'TRONROUT0000000347624445',
 'TRONROUT0000000032852619',
 'TRONROUT0000000032852608',
 'TRONROUT0000000032856211']

In [8]:
at.recup_troncon_parallele_v2(df,toto)

C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\agrege_troncon.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))


'TRONROUT0000000202559739'

In [ ]:
tata=at.affecter_troncon(df)

In [120]:
dico=at.affecter_troncon(df)
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

0eme occurence : TRONROUT0000000084053019 à 09:48:56 nb ligne traite : 0, nb ligne differente=0
erreur index a ligne ligne : TRONROUT0000000032970976
erreur index a ligne ligne : TRONROUT0000000039536310
erreur index a ligne ligne : TRONROUT0000000084123085
erreur index a ligne ligne : TRONROUT0000000084123088
erreur index a ligne ligne : TRONROUT0000000032856229
erreur index a ligne ligne : TRONROUT0000000200796439
erreur index a ligne ligne : TRONROUT0000000089679810
erreur index a ligne ligne : TRONROUT0000000032867128
erreur index a ligne ligne : TRONROUT0000000202559733
erreur index a ligne ligne : TRONROUT0000000202559738
300eme occurence : TRONROUT0000000032831832 à 09:49:15 nb ligne traite : 1724, nb ligne differente=1724
erreur index a ligne ligne : TRONROUT0000000032962977
erreur index a ligne ligne : TRONROUT0000000255628546
erreur index a ligne ligne : TRONROUT0000000032827720
erreur index a ligne ligne : TRONROUT0000000032843225
600eme occurence : TRONROUT0000000032848964 

POUR TRAITEMENT DES VOIES A 2 CHAUSSES : TROUVER PARRALLELE

In [172]:
#liste des troncons (parametre entrant)
liste_troncon=toto

In [ ]:
#conversion en geodf
gdf_lignes=gp.GeoDataFrame(df.loc[liste_troncon], geometry='geom')

In [174]:
#union des geometries
gdf_lignes2=gdf_lignes.unary_union

In [176]:
#centroid de la ligne
xmin,ymin,xmax,ymax=gdf_lignes2.interpolate(0.5, normalized=True).buffer(50).bounds

In [177]:
#donnees de base
gdf_global=gp.GeoDataFrame(df, geometry='geom')

In [178]:
#recherche des lignes proches du centroid
lignes_possibles=gdf_global.cx[xmin:xmax,ymin:ymax]

In [ ]:
lignes_possibles

In [180]:
#uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
ligne_filtres=lignes_possibles.loc[(~lignes_possibles.index.isin(liste_troncon)) & (lignes_possibles['numero']==gdf_lignes.iloc[0]['numero'])]

In [ ]:
#obtenir les distances au centroid
ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))

In [182]:
#garder uniquement la valeur la plus proche du centroid
ligne_filtres.loc[ligne_filtres['distance']==ligne_filtres['distance'].min()].index[0]

'TRONROUT0000000202559739'

POUR TRAITEMENT DES ROND POINTS

In [4]:
#polygoniser les lignes
#creer la gdf
df.crs={'init':'epsg:2154'}

In [147]:
#creer la lsite des rd points selon le critere atmo aura ; creer une gdf avec la liste
liste_rd_points=[t.buffer(0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14]
dico_rd_pt=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point=gp.GeoDataFrame(dico_rd_pt, geometry=liste_rd_points)
gdf_rd_point.crs={'init':'epsg:2154'}
gdf_rd_point.columns=['id_rdpt', 'facteur','geometry'] 
#lameme, mais avec unbuffervers l'interieur, pour pouvoir ensuite ne pas prendre les voies qui ont dans le buffer interieur (i.e dans le cas d'un rond point au dessus d'une 2*2 voie par exemple
liste_rd_points_int=[t.buffer(-0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14]
dico_rd_pt_int=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point_int=gp.GeoDataFrame(dico_rd_pt_int, geometry=liste_rd_points_int)
gdf_rd_point_int.crs={'init':'epsg:2154'}
gdf_rd_point_int.columns=['id_rdpt', 'facteur','geometry'] 

In [149]:
l_dans_p=gp.sjoin(df.drop('id_rdpt', axis=1),gdf_rd_point,op='within')
l_dans_p_int=gp.sjoin(df.drop('id_rdpt', axis=1),gdf_rd_point_int,op='within')
final=l_dans_p.loc[~l_dans_p.index.isin(l_dans_p_int.index.tolist())]

In [171]:
#lignes qui touchent rd points
#1.ligne qui intersectent avec id_rdpt
l_intersct_rdpt=gp.sjoin(df.drop('id_rdpt',axis=1),final.drop('index_right', axis=1), how='inner',op='intersects')

In [176]:
#2.filtre de celle contenue dans le rd points 
l_intersct_rdpt=l_intersct_rdpt.loc[~l_intersct_rdpt.index.isin(l_dans_p.index.tolist())][['id_rdpt','numero_left','nom_voie_g_left']]

In [182]:
#trouver le nb de voies qui intersectent chaque rd point et leur noms. renomer les colonnes
carac_rd_pt=(pd.concat([l_intersct_rdpt.groupby('id_rdpt').numero_left.nunique(),
    l_intersct_rdpt.groupby('id_rdpt')['numero_left'].apply(lambda x: ','.join(set(x)))['nom_voie_g_left'].apply(lambda x: ','.join(set(x)))], axis=1))
carac_rd_pt.columns=['nb_rte_rdpt', 'nom_rte_rdpt']

TypeError: sequence item 1: expected str instance, NoneType found

In [177]:
l_intersct_rdpt.columns

Index(['id_rdpt', 'numero_left', 'nom_voie_g_left'], dtype='object')

In [151]:
carac_rd_pt

,nb_rte_rdpt,nom_rte_rdpt
id_rdpt,,
0,3,"NC,D115,D939BIS"
18,1,D137
24,1,N537
27,1,D116
51,2,"NC,D127"
55,2,"NC,D127"
58,1,D939
61,1,D105
68,1,D114


In [ ]:
#ajouter l'id_rdpt aux données
gdf_l_rd_point=pd.concat([df,final.loc[:,'id_rdpt']],axis=1, sort=False)
#mettre à jour la nature
gdf_l_rd_point['nature']=gdf_l_rd_point.apply(lambda x : 'Rd_pt' if x.id_rdpt>=0 else x['nature'], axis=1)
#ajouter les infos du rd point (nb voies différentes et nom)
gdf_avec_rd_pt=gdf_l_rd_point.merge(carac_rd_pt, how='left',left_on='id_rdpt', right_index=True)
#gdf_avec_rd_pt.loc[gdf_avec_rd_pt.loc[:,'id_rdpt']==l_dans_p['id_rdpt']]
gdf_avec_rd_pt['id_rdpt']==final['id_rdpt'].astype('float64')
#l_dans_p.loc[l_dans_p.loc[:,'id_rdpt']].sort_index().sort_index(axis=1)])

In [ ]:
#pour export et verif uniquement
#gdf_l_rd_point.to_file(r'E:\Boulot\otv\test_linearisation_auto\test2.shp')
gdf_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test1.shp')
gdf_rd_point_int.to_file(r'D:\temp\otv\17\test_rd_point\test6.shp')
#l_intersct_rdpt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test3.shp')
#gdf_avec_rd_pt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test4.shp')
#df.to_file(r'D:\temp\otv\17\test_rd_point\test5.shp')

In [91]:
','.join(set(['a','a','b','c']))

'b,c,a'

In [49]:
id_ign_ligne='TRONROUT0000000032856211'
df_ligne = df.loc[id_ign_ligne]
ligne_traite_troncon=[]
ligne_traite_troncon.append(id_ign_ligne)
liste_ligne_suivantes=[] 
liste_ligne_touchees=[]

In [50]:
for key, value in {'nb_intrsct_src':['source', 'src_geom'],'nb_intrsct_tgt':['target', 'tgt_geom']}.items() :
    df_touches_source = df.loc[(~df.index.isin(ligne_traite_troncon)) & ((df['source'] == df_ligne[value[0]]) | (df['target'] == df_ligne[value[0]]))]
    liste_ligne_touchees+=df_touches_source.index.tolist()

In [51]:
df_touches_source['nom_voie_g'].values.tolist()

['AV JEAN MOULIN']

In [52]:
liste_ligne_touchees

['TRONROUT0000000032856177',
 'TRONROUT0000000032856205',
 'TRONROUT0000000032852549']